По традиции начинаем с вопроса....

У нас есть множество различных новостных статей на абсолютно разные темы. Нам нужно определить тему статьи (узкую) (найти в массе из 3000+ знаков какие-то самые значимые для этой статьи слова, которые делают её отличной от других статей и которые могут поступить в заголовок и составлять её  краткий пересказ) --- как нам это сделать? 


Очевидным образом, не все слова were created equal. Мы уже знакомы с рядос стоп слов, которые мы в лингвистике яростно выбрасываем из предложения, однако, и другие слова в тексте неравнознчны.......

# TF_IDF

Что это? И зачем оно нужно? 

По сути, это метрика, которая показывает насколько важно/релевантно слово в заданном документе. используется для анализа темы и в поисковиках.

В этом семинаре мы попытаемся создать с помощью неё наш собственный поисковик.

Представим, что нам на вход в поисковике поступило 'a brown cow'. У нас в мировой паутине лежит куча-куча-куча документов с множеством различных слов. Как по заданным словам понять, какой из них релевантнее? 

In [38]:
import pandas as pd # если нет pandas - не страшно - он тут для красоты


In [22]:
inquery = 'the brown cows'

documentA = 'the black man went out at night for a stroll past the brown cows he owned He also noticed a wolf on the stroll but did not pay attention ...'
documentB = 'The brown children sat around the fire and sang the songs ...'
documentC = 'Everything about brown cows.  Brown cows live in a harsh enviroment ... '
documentD = 'Everything about cows. Cows live in different enviroments ...'

Как мы можем, не залезая в семантику, узнать, какой текст выдать первым?

tf


idf


term frequency–inverse document frequency

In [23]:

import nltk
nltk.download('punkt')
bagOfWordsA = nltk.word_tokenize(documentA.lower())
bagOfWordsB = nltk.word_tokenize(documentB.lower())
bagOfWordsC = nltk.word_tokenize(documentC.lower())
bagOfWordsD = nltk.word_tokenize(documentD.lower())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avorl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB)).union(set(bagOfWordsC)).union(set(bagOfWordsD))

In [25]:
uniqueWords

{'.',
 '...',
 'a',
 'about',
 'also',
 'and',
 'around',
 'at',
 'attention',
 'black',
 'brown',
 'but',
 'children',
 'cows',
 'did',
 'different',
 'enviroment',
 'enviroments',
 'everything',
 'fire',
 'for',
 'harsh',
 'he',
 'in',
 'live',
 'man',
 'night',
 'not',
 'noticed',
 'on',
 'out',
 'owned',
 'past',
 'pay',
 'sang',
 'sat',
 'songs',
 'stroll',
 'the',
 'went',
 'wolf'}

In [26]:
numOfWordsA = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1

numOfWordsB = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsB:
    numOfWordsB[word] += 1
    
numOfWordsC = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsC:
    numOfWordsC[word] += 1
    
numOfWordsD = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsD:
    numOfWordsD[word] += 1
    
df = pd.DataFrame([numOfWordsA, numOfWordsB,numOfWordsC, numOfWordsD ])    

In [27]:
pd.set_option('display.max_columns', None)
df

,about,in,on,harsh,sang,the,enviroment,stroll,did,...,live,but,a,fire,not,also,he,brown,night,at,sat,children,cows,noticed,songs,out,owned,enviroments,wolf,different,pay,man,and,went,past,for,everything,around,attention,black,.
0,0,0,1,0,0,3,0,2,1,1,0,1,2,0,1,1,2,1,1,1,0,0,1,1,0,1,1,0,1,0,1,1,0,1,1,1,0,0,1,1,0
1,0,0,0,0,1,3,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,1,1,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,2,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1


Почему иногда всё же но стоит выкидывать стоп-слова - предлоги, артикли и союзы?


## давайте построим свой tf-idf

## TF

Tf --- term frequency. В самом первом упрощении - это то, сколько раз слово встретилось в тексте (чем чаще искомое слово встретилось в заданном тексте по сравнению с другими, тем этот текст релевантнее запросу), в чём тут проблема? 

Поэтому есть всякие adjustments. Так часто tf слова в документе считается как количество раз, когда оно появилось в этом документе, деленное на общее число слов документе.

In [28]:
def computeTF(wordDict, bagOfWords):
    #напишите функцию, которая получает на ввход bagofwords и numOfWords, а возвращает словарь с tf-ами для каждого слова из numofwords
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [29]:
tfA = computeTF(numOfWordsA, bagOfWordsA)
tfB = computeTF(numOfWordsB, bagOfWordsB)
tfC = computeTF(numOfWordsC, bagOfWordsC)
tfD = computeTF(numOfWordsD, bagOfWordsD)

In [30]:
tfB

{'about': 0.0,
 'in': 0.0,
 'on': 0.0,
 'harsh': 0.0,
 'sang': 0.08333333333333333,
 'the': 0.25,
 'enviroment': 0.0,
 'stroll': 0.0,
 'did': 0.0,
 '...': 0.08333333333333333,
 'live': 0.0,
 'but': 0.0,
 'a': 0.0,
 'fire': 0.08333333333333333,
 'not': 0.0,
 'also': 0.0,
 'he': 0.0,
 'brown': 0.08333333333333333,
 'night': 0.0,
 'at': 0.0,
 'sat': 0.08333333333333333,
 'children': 0.08333333333333333,
 'cows': 0.0,
 'noticed': 0.0,
 'songs': 0.08333333333333333,
 'out': 0.0,
 'owned': 0.0,
 'enviroments': 0.0,
 'wolf': 0.0,
 'different': 0.0,
 'pay': 0.0,
 'man': 0.0,
 'and': 0.08333333333333333,
 'went': 0.0,
 'past': 0.0,
 'for': 0.0,
 'everything': 0.0,
 'around': 0.08333333333333333,
 'attention': 0.0,
 'black': 0.0,
 '.': 0.0}

# idf

idf --- The log of the number of documents divided by the number of documents that contain the word w. Inverse data frequency determines the weight of rare words across all documents in the corpus.

In [31]:
import math
def computeIDF(documents):
    #для того, чтобы посчитать idf надо подать все документы в функцию сразу 
    #напишите функцию, считающую idf для каждого слова - на входе массив из numOfWords для всех текстов 
    #на выходе словарь для слов 
    #см. ввод и вывод ниже
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents: # 
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [32]:
idfs = computeIDF([numOfWordsA, numOfWordsB, numOfWordsC, numOfWordsD])

In [33]:
idfs

{'about': 0.6931471805599453,
 'in': 0.6931471805599453,
 'on': 1.3862943611198906,
 'harsh': 1.3862943611198906,
 'sang': 1.3862943611198906,
 'the': 0.6931471805599453,
 'enviroment': 1.3862943611198906,
 'stroll': 1.3862943611198906,
 'did': 1.3862943611198906,
 '...': 0.0,
 'live': 0.6931471805599453,
 'but': 1.3862943611198906,
 'a': 0.6931471805599453,
 'fire': 1.3862943611198906,
 'not': 1.3862943611198906,
 'also': 1.3862943611198906,
 'he': 1.3862943611198906,
 'brown': 0.28768207245178085,
 'night': 1.3862943611198906,
 'at': 1.3862943611198906,
 'sat': 1.3862943611198906,
 'children': 1.3862943611198906,
 'cows': 0.28768207245178085,
 'noticed': 1.3862943611198906,
 'songs': 1.3862943611198906,
 'out': 1.3862943611198906,
 'owned': 1.3862943611198906,
 'enviroments': 1.3862943611198906,
 'wolf': 1.3862943611198906,
 'different': 1.3862943611198906,
 'pay': 1.3862943611198906,
 'man': 1.3862943611198906,
 'and': 1.3862943611198906,
 'went': 1.3862943611198906,
 'past': 1.3862

## tf_idf  целиком

тф_айдф для слова в документе - это просто произведение его тф в документе на его идф

In [34]:
def computeTFIDF(tfBagOfWords, idfs):
    #давайте
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [35]:
tfidfA = computeTFIDF(tfA, idfs)
tfidfB = computeTFIDF(tfB, idfs)
tfidfC = computeTFIDF(tfC, idfs)
tfidfD = computeTFIDF(tfD, idfs)
df = pd.DataFrame([tfidfA, tfidfB, tfidfC, tfidfD])

In [36]:
df

,about,in,on,harsh,sang,the,enviroment,stroll,did,...,live,but,a,fire,not,also,he,brown,night,at,sat,children,cows,noticed,songs,out,owned,enviroments,wolf,different,pay,man,and,went,past,for,everything,around,attention,black,.
0,0.000000,0.000000,0.04621,0.000000,0.000000,0.069315,0.000000,0.09242,0.04621,0.0,0.000000,0.04621,0.046210,0.000000,0.04621,0.04621,0.09242,0.009589,0.04621,0.04621,0.000000,0.000000,0.009589,0.04621,0.000000,0.04621,0.04621,0.000000,0.04621,0.000000,0.04621,0.04621,0.000000,0.04621,0.04621,0.04621,0.000000,0.000000,0.04621,0.04621,0.000000
1,0.000000,0.000000,0.00000,0.000000,0.115525,0.173287,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.000000,0.115525,0.00000,0.00000,0.00000,0.023974,0.00000,0.00000,0.115525,0.115525,0.000000,0.00000,0.115525,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.00000,0.115525,0.00000,0.00000,0.00000,0.000000,0.115525,0.00000,0.00000,0.000000
2,0.053319,0.053319,0.00000,0.106638,0.000000,0.000000,0.106638,0.00000,0.00000,0.0,0.053319,0.00000,0.053319,0.000000,0.00000,0.00000,0.00000,0.044259,0.00000,0.00000,0.000000,0.000000,0.044259,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.053319,0.000000,0.00000,0.00000,0.053319
3,0.069315,0.069315,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.069315,0.00000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.057536,0.00000,0.000000,0.00000,0.00000,0.138629,0.00000,0.138629,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.069315,0.000000,0.00000,0.00000,0.069315


посмотрим на brown

## Автоматизация

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA, documentB, documentC, documentD])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [40]:
df

,about,also,and,around,at,attention,black,brown,but,children,cows,did,different,enviroment,enviroments,everything,fire,for,harsh,he,in,live,man,night,not,noticed,on,out,owned,past,pay,sang,sat,songs,stroll,the,went,wolf
0,0.000000,0.175657,0.000000,0.000000,0.175657,0.175657,0.175657,0.112120,0.175657,0.000000,0.112120,0.175657,0.000000,0.000000,0.000000,0.000000,0.000000,0.175657,0.000000,0.351315,0.000000,0.000000,0.175657,0.175657,0.175657,0.175657,0.175657,0.175657,0.175657,0.175657,0.175657,0.000000,0.000000,0.000000,0.351315,0.415471,0.175657,0.175657
1,0.000000,0.000000,0.277331,0.277331,0.000000,0.000000,0.000000,0.177017,0.000000,0.277331,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.277331,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.277331,0.277331,0.277331,0.000000,0.655954,0.000000,0.000000
2,0.283285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.458688,0.000000,0.000000,0.458688,0.000000,0.000000,0.359311,0.000000,0.283285,0.000000,0.000000,0.359311,0.000000,0.283285,0.283285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.318800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.516193,0.000000,0.404358,0.000000,0.404358,0.318800,0.000000,0.000000,0.000000,0.000000,0.318800,0.318800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Теперь даваайте создадим наш парсер

In [46]:
import csv
import pymorphy2
import string
import re
from math import log
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

morph = pymorphy2.MorphAnalyzer()
vectorizer1 = CountVectorizer(binary = True)
vectorizertf = TfidfVectorizer(use_idf=False)

['', 'question1', 'question2', 'is_duplicate']

['0', 'Какова история кохинор кох-и-ноор-бриллиант', 'что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад', '0']

['1', 'как я могу увеличить скорость моего интернет-соединения, используя vpn', 'как повысить скорость интернета путем взлома через dns', '0']

['2', 'почему я мысленно очень одинок, как я могу это решить', 'найти остаток, когда математика 23 ^ 24 математика разделена на 24 23', '0']

['3', 'которые растворяют в воде быстро сахарную соль метан и углеродный диоксид', 'какая рыба выживет в соленой воде', '0']

['4', 'астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне', 'Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне', '1']

давайте обработаем только  20 000

In [53]:
def get_texts(): # возвращаем [corpus, inquiery, lenghths, scores, normal]
    corpus = [] #сюда кладём тексты из [2] столбца, где каждое слово прошло препроцессинг
                #(нижний регистр, нет знаков препинания) и каждое слово в начальной форме
    inquiery = [] #сюда кладём тексты из [1] столбца - их препроцесить не надо
    #lenghths = []
    scores = [] #сюда кладём информацию из [3] -- является ли ответ соответсвующим вопросу
    normal = []

    with open('quora_question_pairs_rus.csv', encoding = 'utf-8') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            # отсюда убрать
            normal.append(row[2])
            text = preproc(row[2])
            corpus.append(text)
            inquiery.append(row[1])
            #lenghths.append(len(text.split(' ')))
            scores.append(row[3])
            if len(scores) == 20000: #оч плохо
                #return [corpus, inquiery, lenghths, scores, normal]
                return [corpus, inquiery, scores, normal]

def preproc(data):                      # препроцессинг (убираем знаки препинания и числа, приводим всё к начальной форме)
            data = data.split()        
            text = ''                   # числа есть только в тех информации, в самом скрипте все числа пишутся буквами
            for word in data:
                word = word.strip('[!,.?"]')
                p = morph.parse(word.strip())[0]
                p = p.normal_form
                if p != '-':
                    if re.search(r'\d', p) == None:
                        text  = text + ' ' + p
            return text[1:]    

In [56]:
data = get_texts()

In [58]:
corpus_texts, inquiery_texts, scores, normal_texts =  data[0][1:], data[1][1:], data[2][1:], data[3][1:]
del data

In [59]:
print(corpus_texts[:5])
print(inquiery_texts[:5])
print(scores[:5])

['что произойти если правительство индия украсть кохинор кох-и-ноор-алмаз назад', 'как повысить скорость интернет путём взлом через dns', 'найти остаток когда математик ^ математик разделить на', 'какой рыба выжить в солёный вода', 'я тройной луна-козерог и восхождение в козерог что это говорить о я']
['Какова история кохинор кох-и-ноор-бриллиант', 'как я могу увеличить скорость моего интернет-соединения, используя vpn', 'почему я мысленно очень одинок, как я могу это решить', 'которые растворяют в воде быстро сахарную соль метан и углеродный диоксид', 'астрология: я - луна-колпачок из козерога и крышка, поднимающая то, что это говорит обо мне']
['0', '0', '0', '0', '1']


In [60]:
corpus_matrix1 = vectorizer1.fit_transform(corpus_texts)
voc = vectorizer1.get_feature_names() #получили вокабуляр -- какие вообще есть слова, у каждого еще и свой индекс

In [61]:
tf = vectorizertf.fit_transform(corpus_texts)
tf = tf.toarray()

In [80]:
def search_mat( inquiery, voc):
    #делаем препроцессинг инквери
    inq = preproc(inquiery)
    # создали вектор из нулей, равный нашему вакабуляру
    vec = np.zeros((len(voc)))
    for word in inq.split(' '):
        if word in voc:
            index = voc.index(word)
            vec[index] = 1 # нули начинают заполняться единицами
    res = np.dot(tf,vec) #перемножили 2 вектора --- получили что?
    
    results = np.argsort(res)[::-1][:5]
    
   
    for i in results:
        print(normal_texts[i])
    return

In [81]:
search_mat('правительство индии',voc)


как работает правительство Индии
почему правительство Индии запрещает все
какая разница между парламентом и правительством Индии
есть ли квота в Индии в Индии?
каковы преимущества решения правительства Индии о вывозе 500 и 1000 рупий
